# Housing Prices Prediction

## Import Libraries

In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Import Dataset

In [33]:
train_set = pd.read_csv("data/train.csv")
test_set = pd.read_csv("data/test.csv")

In [34]:
train_set.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [35]:
test_set.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


## Clean Dataset and Preprocessing

### Split Numerical and Categorical Variables

In [36]:
num_feat = train_set.select_dtypes(include=['float', 'int'])
cat_feat = train_set.select_dtypes(include=['object'])

In [37]:
print(len(num_feat.columns) + len(cat_feat.columns))

81


### Impute Null Values and Drop Unecessary Columns

In [38]:
# Examine null values for numerical data
missing_num = num_feat.isnull().sum()
nan_num = missing_num[missing_num > 0]
nan_num

LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64

In [39]:
# Examine null values for categorical data
missing_cat = cat_feat.isnull().sum()
nan_cat = missing_cat[missing_cat > 0]
nan_cat

Alley           1369
MasVnrType         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [40]:
def feat_null_pct(num):
    missing = num.isnull().sum()
    nans = missing[missing > 0].sort_values(ascending=False)
    for col in nans.index:
        print(f"{col} Null Values: {round(100 - train_set[col].value_counts().sum()/len(train_set) * 100, 2)}% of the dataset")

feat_null_pct(num_feat)

LotFrontage Null Values: 17.74% of the dataset
GarageYrBlt Null Values: 5.55% of the dataset
MasVnrArea Null Values: 0.55% of the dataset


In [41]:
feat_null_pct(cat_feat)

PoolQC Null Values: 99.52% of the dataset
MiscFeature Null Values: 96.3% of the dataset
Alley Null Values: 93.77% of the dataset
Fence Null Values: 80.75% of the dataset
FireplaceQu Null Values: 47.26% of the dataset
GarageType Null Values: 5.55% of the dataset
GarageFinish Null Values: 5.55% of the dataset
GarageQual Null Values: 5.55% of the dataset
GarageCond Null Values: 5.55% of the dataset
BsmtExposure Null Values: 2.6% of the dataset
BsmtFinType2 Null Values: 2.6% of the dataset
BsmtQual Null Values: 2.53% of the dataset
BsmtCond Null Values: 2.53% of the dataset
BsmtFinType1 Null Values: 2.53% of the dataset
MasVnrType Null Values: 0.55% of the dataset
Electrical Null Values: 0.07% of the dataset


In [42]:
# Drop the columns with large proportions of Null Values and Unique Id
train_set.drop(['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence'], axis=1, inplace=True)
test_set.drop(['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence'], axis=1, inplace=True)

# Update numerical and categorical variable split
num_feat = train_set.select_dtypes(include=['int', 'float'])
cat_feat = train_set.select_dtypes(include=['object'])

num_feat_test = test_set.select_dtypes(include=['int', 'float'])
cat_feat_test = test_set.select_dtypes(include=['object'])

In [43]:
num_feat.shape

(1460, 37)

In [44]:
cat_feat.shape

(1460, 39)

In [45]:
from sklearn.impute import SimpleImputer

# Impute Numerical Columns
imputer_num = SimpleImputer(missing_values=np.nan, strategy='mean')
num_feat = pd.DataFrame(imputer_num.fit_transform(num_feat), columns=num_feat.columns)

imputer_num_test = SimpleImputer(missing_values=np.nan, strategy='mean')
num_feat_test = pd.DataFrame(imputer_num.fit_transform(num_feat_test), columns=num_feat_test.columns)

In [46]:
cat_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
cat_feat = pd.DataFrame(cat_imputer.fit_transform(cat_feat), columns=cat_feat.columns)

cat_imputer_test = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
cat_feat_test = pd.DataFrame(cat_imputer.fit_transform(cat_feat_test), columns=cat_feat_test.columns)

In [47]:
print(f"Null Numerical Data: {num_feat.isnull().any().sum() + num_feat_test.isnull().any().sum()}")
print(f"Null Categorical Data: {cat_feat.isnull().any().sum() + cat_feat_test.isnull().any().sum()}")

Null Numerical Data: 0
Null Categorical Data: 0


### Encoding Categorical Data

In [48]:
# Concatenating the training and test categorical features to avoid mismatching the columns
cat_concat = pd.concat([cat_feat, cat_feat_test], axis=0)

In [49]:
print(cat_feat.shape, cat_feat_test.shape, cat_concat.shape)

(1460, 39) (1459, 39) (2919, 39)


In [50]:
cat_concat.columns

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object')

In [51]:
# Encode categorical columns while avoiding dummy variable trap
def get_dummy_no_trap(cat_data):
    copy_cat = cat_data.copy()
    cat_out = cat_data
    first = True
    for col in copy_cat.columns:
        temp_cat = pd.get_dummies(copy_cat[col], drop_first=True)
        copy_cat.drop([col], axis=1, inplace=True)
        if first:
            cat_out = temp_cat.copy()
            first = False
        else:
            cat_out = pd.concat([cat_out, temp_cat], axis=1)
    cat_out = pd.concat([copy_cat, cat_out], axis=1)
    
    return cat_out

In [52]:
cat_copy = cat_concat.copy()

In [53]:
encoded_cat = get_dummy_no_trap(cat_copy)

In [54]:
encoded_cat.shape

(2919, 200)

In [55]:
encoded_cat.head()

,FV,RH,RL,RM,Pave,IR2,IR3,Reg,HLS,Low,...,ConLI,ConLw,New,Oth,WD,AdjLand,Alloca,Family,Normal,Partial
0,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
2,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [56]:
print(encoded_cat.columns)

Index(['FV', 'RH', 'RL', 'RM', 'Pave', 'IR2', 'IR3', 'Reg', 'HLS', 'Low',
       ...
       'ConLI', 'ConLw', 'New', 'Oth', 'WD', 'AdjLand', 'Alloca', 'Family',
       'Normal', 'Partial'],
      dtype='object', length=200)


In [57]:
# Resplit the rows
cat_feat = encoded_cat.iloc[:1460]
cat_feat_test = encoded_cat.iloc[1460:]

In [58]:
cat_feat

,FV,RH,RL,RM,Pave,IR2,IR3,Reg,HLS,Low,...,ConLI,ConLw,New,Oth,WD,AdjLand,Alloca,Family,Normal,Partial
0,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
2,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1456,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1457,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1458,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0


In [59]:
cat_feat_test

,FV,RH,RL,RM,Pave,IR2,IR3,Reg,HLS,Low,...,ConLI,ConLw,New,Oth,WD,AdjLand,Alloca,Family,Normal,Partial
0,0,1,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0,0,1,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0,0,0,1,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1455,0,0,0,1,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1456,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1457,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0


## Finalize the Dataset

In [60]:
# Recombine training set
train_set = pd.concat([num_feat, cat_feat], axis=1)
train_set = train_set.loc[:, ~train_set.columns.duplicated()]

# Recombine test set
test_set = pd.concat([num_feat_test, cat_feat_test], axis=1)
test_set = test_set.loc[:, ~test_set.columns.duplicated()]

In [61]:
train_set.shape

(1460, 177)

In [62]:
test_set.shape

(1459, 176)

## Implement XGBoost Regressor

In [64]:
X_train = train_set.drop(['SalePrice'], axis=1)
y_train = train_set['SalePrice']

In [65]:
X_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,ConLI,ConLw,New,Oth,WD,AdjLand,Alloca,Family,Normal,Partial
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,0,0,0,0,1,0,0,0,1,0
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,0,0,0,0,1,0,0,0,1,0
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,0,0,0,0,1,0,0,0,1,0
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,62.0,7917.0,6.0,5.0,1999.0,2000.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0
1456,20.0,85.0,13175.0,6.0,6.0,1978.0,1988.0,119.0,790.0,163.0,...,0,0,0,0,1,0,0,0,1,0
1457,70.0,66.0,9042.0,7.0,9.0,1941.0,2006.0,0.0,275.0,0.0,...,0,0,0,0,1,0,0,0,1,0
1458,20.0,68.0,9717.0,5.0,6.0,1950.0,1996.0,0.0,49.0,1029.0,...,0,0,0,0,1,0,0,0,1,0


In [66]:
y_train

0       208500.0
1       181500.0
2       223500.0
3       140000.0
4       250000.0
          ...   
1455    175000.0
1456    210000.0
1457    266500.0
1458    142125.0
1459    147500.0
Name: SalePrice, Length: 1460, dtype: float64

In [67]:
from xgboost import XGBRegressor
xgb_reg = XGBRegressor()

n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]
base_score=[0.25,0.5,0.75,1]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [68]:
from sklearn.model_selection import RandomizedSearchCV

# Set up the random search with 5-fold cross validation
random_cv = RandomizedSearchCV(estimator=xgb_reg,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [69]:
random_cv.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          predictor=None, random_state=None, ...),
                   n_iter=50, n_jobs=4,
                   param_distributions={'base_score': [0.25, 0.5, 0.75, 1],
                                        'booster': ['gbtree', 'gblinear'],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2],
                                        'max_depth': [2, 3, 5, 10, 15],
                                        'min_child_weight': [1, 2, 3, 4],
                                        'n_estimators': [100, 500, 900, 1100,
                                                         1500]},
                   random_state=42, return_train_score=True,
                   scoring='neg_mean_absolute_error', verbose=5)

In [71]:
best_model = random_cv.best_estimator_
y_pred = best_model.predict(test_set)

## Reformat Prediction

In [72]:
y_pred

array([117704.984, 161415.08 , 185766.23 , ..., 176330.12 , 119072.57 ,
       233318.95 ], dtype=float32)

In [75]:
# Append Id column to y_pred
id_col = pd.read_csv('data/sample_submission.csv')['Id']
final_submission = pd.DataFrame({'Id' : id_col, 'SalePrice' : y_pred})

In [76]:
final_submission

,Id,SalePrice
0,1461,117704.984375
1,1462,161415.078125
2,1463,185766.234375
3,1464,194221.531250
4,1465,197895.046875
...,...,...
1454,2915,86123.687500
1455,2916,67338.289062
1456,2917,176330.125000
1457,2918,119072.570312


In [77]:
# Save in csv file
final_submission.to_csv("data/submission.csv", index=False)